# 데이터 불러오기

In [1]:
! pip3 install koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 63.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [11]:
application_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Application.csv"
company_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Company.csv"
job_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Job.csv"
job_bookmark_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/JobBookmark.csv"

In [4]:
application_df = pd.read_csv(application_path)
application_df.head(2)

,Unnamed: 0,cdate,company_uuid,job_uuid,user_uuid,application_uuid
0,0,2017-02-25 23:45:01,de4b3596-b4ab-47cd-b8ea-6f9e14170064,459e461d-a571-4ed3-8751-8f4cb5b9ff37,b0329bd7-fc45-4e83-993a-b73bd9d3549c,59c42363-f764-4cbd-aafd-20eb663ea5d7
1,1,2020-11-23 14:46:07.329731,f6156b12-d4d7-469f-84be-31799c765211,e5ed4f4a-08aa-4f2d-9042-1e9bbd36803f,b0329bd7-fc45-4e83-993a-b73bd9d3549c,425db0e8-87f8-4b4d-8248-7f24c31d49d3


In [12]:
company_df = pd.read_csv(company_path)
company_df.head(2)

,Unnamed: 0,cdate,mdate,found_date,employee_count,view_count,follow_count,reference_count,company_uuid
0,0,2022-06-09 04:08:40,2022-06-09 04:08:40,NaN,0명,0,0,0,c46f2fa5-f940-40fb-bd4a-aa0ff7a837fd
1,1,2017-05-22 10:57:12,2022-07-25 00:12:02,NaN,0명,256,0,0,725e87bb-de2f-416a-a6b8-1ca8d0d096e0


In [5]:
job_df = pd.read_csv(job_path)
job_df.head(2)

,Unnamed: 0,cdate,mdate,job_field,career_type_string,start_date,end_date,allow_remote,can_show_salary,job_uuid,company_uuid
0,0,2020-11-25 10:32:10,2020-11-25 10:32:10,SW 개발,"신입,경력,인턴",NaN,2020-11-25,1,0,764292b0-53f8-4f14-ac83-6eafc7a6a18d,daa34559-fc42-47f6-b5c6-cc6171404f37
1,1,2020-12-03 16:24:16,2020-12-03 16:24:16,HW 개발,인턴,2020-12-03,2020-12-04,1,0,b2b598d6-9ca3-4eb4-ac9c-bb57a6dad2ad,daa34559-fc42-47f6-b5c6-cc6171404f37


In [8]:
job_bookmark_df = pd.read_csv(job_bookmark_path)
job_bookmark_df.head(2)

# 채용 성공률

- 지원자 수 대비 조회수, 북마크 수 등을 통해 유저의 관심 -> 행동 전환 비율로 추정할 수 있음
- 핵심 비율 정의 (가정 기반)
    - 지원 전환률 (지원수 / 조회수)
    - 북마크 전환률 (지원수 / 북마크수)
    - 공고당 평균 지원자 수
    - 직무/경력/회사별 평균 전환률

In [15]:
# 지원 데이터 + 채용공고
application_job_df = pd.merge(application_df, job_df, on="job_uuid", how="left")

# 북마크 수
bookmark_counts = job_bookmark_df.groupby("job_uuid").size().reset_index(name="bookmark_count")

# 조회수는 job_df에 없으므로 company_df의 view_count를 대신 사용할 수 있음 (대체안)
job_with_bookmarks = pd.merge(job_df, bookmark_counts, on="job_uuid", how="left")
job_with_bookmarks["bookmark_count"] = job_with_bookmarks["bookmark_count"].fillna(0)

# 지원 수
application_counts = application_df.groupby("job_uuid").size().reset_index(name="application_count")

# 최종 병합
job_stats = job_with_bookmarks.merge(application_counts, on="job_uuid", how="left")
job_stats["application_count"] = job_stats["application_count"].fillna(0)

## 1. 전환율 계산 및 직무/경력 분석

In [16]:
# 북마크 전환율
job_stats["bookmark_conversion_rate"] = (job_stats["application_count"] / job_stats["bookmark_count"]) * 100
job_stats["bookmark_conversion_rate"] = job_stats["bookmark_conversion_rate"].replace([np.inf, -np.inf], np.nan)

# 직무별 평균 전환율
by_job_field = job_stats.groupby("job_field")["bookmark_conversion_rate"].mean().sort_values(ascending=False)

# 경력 조건별 전환율
by_career = job_stats.groupby("career_type_string")["bookmark_conversion_rate"].mean().sort_values(ascending=False)

# 회사별 전환율
job_stats = pd.merge(job_stats, company_df[["company_uuid", "view_count"]], on="company_uuid", how="left")
company_success = job_stats.groupby("company_uuid")["bookmark_conversion_rate"].mean().sort_values(ascending=False)

In [17]:
print("🔍 직무별 평균 지원 전환율 (북마크 기준):")
print(by_job_field.head(10))

print("\n👔 경력 조건별 전환율:")
print(by_career)

print("\n🏢 전환율 높은 상위 기업:")
print(company_success.head(10))

🔍 직무별 평균 지원 전환율 (북마크 기준):
job_field
SW 개발    122.799652
투자       104.403215
경영지원      87.847824
게임 개발     83.324186
비즈니스      73.347002
HW 개발     71.640379
디자인       68.442395
기획/PM     58.552593
마케팅       57.826208
운영        54.386296
Name: bookmark_conversion_rate, dtype: float64

👔 경력 조건별 전환율:
career_type_string
경력          96.113226
신입,경력       87.421759
신입          80.860066
신입,경력,인턴    73.742970
신입,인턴       58.490580
경력,인턴       54.558974
인턴          47.412742
Name: bookmark_conversion_rate, dtype: float64

🏢 전환율 높은 상위 기업:
company_uuid
035daeb0-b8eb-4fb1-bb06-f8b9008e8c18    1200.0
51e7ebaa-fdd6-403a-b1eb-a6dbd17cea47    1100.0
de71340a-b6b0-44e8-9121-73f2c6cbb8b6    1100.0
f94c1f3a-7768-46ea-9046-0bf46781cfbc    1000.0
4cdccb5f-d7d9-4c03-ac6f-5ff96bf20691    1000.0
8062bb36-cf50-4312-9535-ff5647a56a74    1000.0
f7bafdc6-55eb-424c-9591-9b7746f0d46b     900.0
9a66b2e6-2c1b-4220-bd85-a847e86b12b0     900.0
eaf8d02c-5488-41de-9300-34773c15b2aa     800.0
ea91b2f3-c6d6-4d47-bb94-1a6cc

- 💡 활용 아이디어:
    - 전환율 높은 직무/경력 조건은 향후 추천 채용공고 최적화, 타겟 알림, 자동 푸시 전략에 활용
    - 전환율 높은 기업은 파트너십 우선, 혹은 사례 인터뷰, 브랜딩 강화
    - 전환율 낮은 기업/직무는 UX 또는 공고 퀄리티 측면에서 개선 유도 메시지 가능